# Validation and correction

As we've continued working with the scraped data versus the changes we've made from the initial csv files given to the LRC. We've done validation and corrections to the initial csv files that are not reflected in the scraped datasets. So now since we've uploaded our data into SQLite the next step will be to cross validate and update the SQLite. Which will be done below

In [6]:
## setup and imports
import os
import re
import csv
import sqlite3
import hashlib
import glob
from pathlib import Path
from datetime import datetime
from collections import defaultdict, Counter
from typing import List, Dict, Tuple, Optional, Union, Any

# Data processing
import pandas as pd
import numpy as np

note the original local datasets given to the LRC initially are hosted in the nahuatl_processing.db under sqLiteDb specifically under the table name: checkpoint_after_empty_p_tag_removal_20251002. For the scraped data, that is under scrapedDataDb under the nahuatl.db file. The schema is under config/ 

In [14]:
scraped_data_dir = Path("../../../data/scrapedDataDb/")
local_data_dir = Path("../../../data/sqLiteDb/")

# Database connection
scraped_db = sqlite3.connect(scraped_data_dir / "nahuatl.db")
local_db = sqlite3.connect(local_data_dir / "nahuatl_processing.db")

# read in the table(s), for the local_db it's only one table (actually two one for the WHP dataset and one for the IDIEZ dataset) while for the 
# scraped_db there are multiple tables due to the relationl structure we want to keep
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", local_db)
WHP_table_name = "checkpoint_after_empty_p_tag_removal_20251002"
IDIEZ_table_name = "IDIEZ_modern_nahuatl-all-2024-03-27T09-45-31"

for me it methodically makes sense to go down the tables that nahuat.db has (check config/schema.sql) and then check each one against the WHP_table_name and the IDIEZ_table_name as such let's begin with the smallest tables first and build up from the tables. 

it would also be smart that as we cross validate we proceed to investigate where the cross references columns actually come from